# Testing Pauli Decomposition

We have computed the Pauli Decomposition of the Parent Hamiltonian for the testing ansatz using **MPS**. Here we compare the Pauli decomposition from **MPS** methods adn state vector using **myQLM**

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
mps_folder = "/home/cesga/gferro/Codigo/TensorNetworks/Zalo/"
myqlm_folder = "/home/cesga/gferro/Codigo/WP3_Benchmark/tnbs/BTC_04_PH/configuration_files/"

In [3]:
mps_list = list(filter(lambda x: x.find("pauli")!=-1,os.listdir(mps_folder)))
mps_list.sort()
myqlm_list = list(filter(lambda x: x.find("pauli")!=-1,os.listdir(myqlm_folder)))
myqlm_list.sort()

## 1. Bare Comparisons Using np.isclose

First bare comparison consist on:

* Load Both decompositions.
* Compare **PauliCoefficients** using np.isclose without no conf. 
* Posible Outpus:
    * 0 -> np.isclose returns a True, here we consider that both decompositions are the same.
    * -1 -> np.isclose returns a False. Decompositions are not the same. There are difference between **PauliCoefficients**. We need to do more comparisons
    * -2. Here there is a problem: The number of Pauli decompositions are diffeent. We need to do more tests.

In [4]:
def compare(mps_name, myqlm_name):
    mps = pd.read_csv(mps_name, sep=";", index_col=0)
    myqlm = pd.read_csv(myqlm_name, sep=";", index_col=0)
    #ps = mps["PauliStrings"] == myqlm["PauliStrings"].all()
    try:
        test = np.isclose(mps["PauliCoefficients"], myqlm["PauliCoefficients"]).all()
        if test == True:
            return 0
        else:
            return -1
    except ValueError:
        return -2
    return test

In [5]:
mps_name = mps_folder + mps_list[0]
myqlm_name = myqlm_folder + myqlm_list[0]
print(mps_name, myqlm_name)

/home/cesga/gferro/Codigo/TensorNetworks/Zalo/nqubits_03_depth_1_pauli.csv /home/cesga/gferro/Codigo/WP3_Benchmark/tnbs/BTC_04_PH/configuration_files/nqubits_03_depth_1_pauli.csv


In [6]:
failed = []
error = []
for mps_, mqylm_ in zip(mps_list, myqlm_list):
    if mps_ == mqylm_:
        mps_name = mps_folder + mps_
        myqlm_name = myqlm_folder + mqylm_
        test = compare(mps_name, myqlm_name)
        if test == -1:
            failed.append(mps_)
        if test == -2:
            error.append(mps_)          
    else:
        raise ValueError("Problem")

In [7]:
print("TotaL : {}. Failed: {}. Error:{}".format(
    len(mps_list), len(failed), len(error)))

TotaL : 112. Failed: 60. Error:27


###  First Results;

**TotaL : 112. Failed: 60. Error:27**

25 decompositions are equal!!

## 2. Comparisons of code -1

Here we are going to compare the difference between the **PauliCoefficients** of the tfiles that returns a -1 in the bare comparison.  In these files the number of Pauli decompoisitions are the same but not all coefficentes can be considered equal usin np.isclose. 

In this case we are goint to execute several np.isclose and we are going to obtain the absolute tolerance for which the coefficients are considered equal by the np.isclose!

In [8]:
def compare_failed(mps_name, myqlm_name):
    mps = pd.read_csv(mps_name, sep=";", index_col=0)
    myqlm = pd.read_csv(myqlm_name, sep=";", index_col=0)
    tols = [1.0e-7, 1.0e-6, 1.0e-5, 1.0e-4, 1.0e-3, 1.0e-2]
    stop = False
    i = 0
    while stop == False:
        #print(i, tols[i])
        index = np.isclose(
            mps["PauliCoefficients"], myqlm["PauliCoefficients"], atol=tols[i]
        )    
        if index.all() == True:
            min_tol = tols[i]
            stop = True
        else:
            i = i + 1 
            if i > len(tols):
                print("PROBLEM")
                stop = True
                min_tol = -1
    return min_tol

In [9]:
tol = []
for fail in failed:
    mps_name = mps_folder + fail
    myqlm_name = myqlm_folder + fail
    tol.append([compare_failed(mps_name, myqlm_name), fail])
tols = pd.DataFrame(tol)

In [10]:
a = tols[[0]].value_counts()

In [11]:
a.sort_index()

0.000001    16
0.000010    18
0.000100    19
0.001000     7
dtype: int64

Changing absolute tolerance for np.isclose we find:

* 0.000001    16
* 0.000010    10
* 0.000100    27
* 0.001000     7

We find that more than 26 files have an aboslute error lower than $10^{-4}$ for all the **Pauli Coefficients** For these case we consider that this coefficients can be considered 0 so we can consider these files equal

In [12]:
print("Total Failed: {}. Tolerance Higher than 1.0e-4: {}".format(
    len(tols), len(tols[tols[0] >= 1.0e-4])
))

Total Failed: 60. Tolerance Higher than 1.0e-4: 26


### Stats on Failed

We have 34 files where the difference between Pauli Coeficients are equal or higher than $10^{-4}$ for any Paulio Coefficient. 

**Can we considered this coefficients equal???** We are going to do more statistics.

In [13]:
def stats_failed(mps_name, myqlm_name, name, tol = 1.0e-4):
    mps = pd.read_csv(mps_name, sep=";", index_col=0)
    myqlm = pd.read_csv(myqlm_name, sep=";", index_col=0)

    error = np.abs(np.abs(mps["PauliCoefficients"]) - np.abs(myqlm["PauliCoefficients"]))
    error_pdf = pd.DataFrame(error.describe()).T
    error_pdf["name"] = name
    error_pdf["9"] = error.quantile(0.95)
    return error_pdf

In [14]:
for_stats = tols[tols[0] >= 1.0e-4]

In [15]:
stats = []
for name in for_stats[1]:
    mps_name = mps_folder + name
    myqlm_name = myqlm_folder + name
    stats.append(stats_failed(mps_name, myqlm_name, name))
stats = pd.concat(stats)
stats.reset_index(drop=True, inplace=True)

In [16]:
stats["75%"].max()

0.00010575667210537665

In [17]:
stats["max"].max()

0.0005474483699702325

In general 75% of the Pauli coefficients has an absolute error lower than $2 * 10 ^{-4}$.

And the highest absolute diference is 0.0005

### Resume

I think it is good to consider thar the Pauli decomosition are equal in both methods

## 2. Errors

The error (-2 code) are related with the number of coefficients presented in the decomposition. In general thes discrepancies are due to the truncation of coeffcients; we use the precision of the float for considering 0.0. Due to problems of resolutions problems acan be arised. Here we analyze 

In [18]:
error

['nqubits_03_depth_1_pauli.csv',
 'nqubits_04_depth_1_pauli.csv',
 'nqubits_05_depth_1_pauli.csv',
 'nqubits_06_depth_1_pauli.csv',
 'nqubits_07_depth_1_pauli.csv',
 'nqubits_08_depth_1_pauli.csv',
 'nqubits_09_depth_1_pauli.csv',
 'nqubits_10_depth_1_pauli.csv',
 'nqubits_11_depth_1_pauli.csv',
 'nqubits_12_depth_1_pauli.csv',
 'nqubits_13_depth_1_pauli.csv',
 'nqubits_14_depth_1_pauli.csv',
 'nqubits_15_depth_1_pauli.csv',
 'nqubits_16_depth_1_pauli.csv',
 'nqubits_17_depth_1_pauli.csv',
 'nqubits_18_depth_1_pauli.csv',
 'nqubits_19_depth_1_pauli.csv',
 'nqubits_21_depth_1_pauli.csv',
 'nqubits_22_depth_1_pauli.csv',
 'nqubits_23_depth_1_pauli.csv',
 'nqubits_24_depth_1_pauli.csv',
 'nqubits_25_depth_1_pauli.csv',
 'nqubits_26_depth_1_pauli.csv',
 'nqubits_27_depth_1_pauli.csv',
 'nqubits_28_depth_1_pauli.csv',
 'nqubits_29_depth_1_pauli.csv',
 'nqubits_30_depth_1_pauli.csv']

In [19]:
bad = []
for fail in error:
    mps_name = mps_folder + fail
    myqlm_name = myqlm_folder + fail
    mps = pd.read_csv(mps_name, sep=";", index_col=0)
    myqlm = pd.read_csv(myqlm_name, sep=";", index_col=0)
    pdf = pd.merge(mps, myqlm, on=["PauliStrings"],how="outer", indicator=True)
    pdf = pdf.dropna()
    test = np.isclose(pdf["PauliCoefficients_x"], pdf["PauliCoefficients_y"]).all()    
    if test != True:
        bad.append(fail)

In [20]:
bad

['nqubits_22_depth_1_pauli.csv',
 'nqubits_23_depth_1_pauli.csv',
 'nqubits_24_depth_1_pauli.csv',
 'nqubits_25_depth_1_pauli.csv',
 'nqubits_26_depth_1_pauli.csv',
 'nqubits_27_depth_1_pauli.csv',
 'nqubits_29_depth_1_pauli.csv',
 'nqubits_30_depth_1_pauli.csv']

In [23]:
name = bad[3]
name

'nqubits_25_depth_1_pauli.csv'

In [22]:
mps_name = mps_folder + name
myqlm_name = myqlm_folder + name
mps = pd.read_csv(mps_name, sep=";", index_col=0)
myqlm = pd.read_csv(myqlm_name, sep=";", index_col=0)

In [24]:
mps["PauliCoefficients"].hist(bins=20, range=(-0.5, 0.5))

<AxesSubplot:>

In [25]:
mps[mps["PauliCoefficients"].abs() > 1.0e-3]

,PauliCoefficients,PauliStrings,Qbits
0,0.500000,III,"[0, 1, 2]"
12,-0.461940,IZI,"[0, 1, 2]"
55,-0.135299,ZXZ,"[0, 1, 2]"
59,0.135299,ZYZ,"[0, 1, 2]"


In [26]:
myqlm[myqlm["PauliCoefficients"].abs() > 1.0e-3]

,PauliCoefficients,PauliStrings,Qbits
0,0.125000,III,"[0, 1, 2]"
1,0.017350,IIX,"[0, 1, 2]"
2,-0.017413,IIY,"[0, 1, 2]"
3,-0.122056,IIZ,"[0, 1, 2]"
4,-0.022956,IXI,"[0, 1, 2]"
...,...,...,...
59,0.033825,ZYZ,"[0, 1, 2]"
60,0.080804,ZZI,"[0, 1, 2]"
61,0.015331,ZZX,"[0, 1, 2]"
62,-0.015390,ZZY,"[0, 1, 2]"


In [ ]:
myqlm.hist(bins=20, range=(-0.5, 0.5))

In [ ]:
%matplotlib inline

In [ ]:
mps["PauliCoefficients"].value_counts()

In [ ]:
myqlm["PauliCoefficients"].value_counts()

In [ ]:
to_test = []
buf = []
for fail in error:
    mps_name = mps_folder + fail
    myqlm_name = myqlm_folder + fail
    mps, myqlm = compared_error(mps_name, myqlm_name, fail)
    pdf = pd.merge(
        mps, myqlm, on = ["PauliStrings"], how="outer", indicator=True
    )
    pdf = pdf[pdf["_merge"] != "both"]
    left = pdf["PauliCoefficients_x"].dropna().abs()
    right = pdf["PauliCoefficients_y"].dropna().abs()
    all_ = pd.concat([right, left], axis=0).mean()
    print(all_)
    if all_ < 1.0e-10:
        to_test.append(fail)
        print("Ol Rait")
    else:
        print("Uoh!!")
        buf.append(fail)
    


In [ ]:
to_test

In [ ]:
buf

In [ ]:
mps_name = mps_folder + to_test[0]
myqlm_name = myqlm_folder + to_test[0]
mps = pd.read_csv(mps_name, sep=";", index_col=0)
myqlm = pd.read_csv(myqlm_name, sep=";", index_col=0)

In [ ]:
len(mps), len(myqlm)

In [ ]:
pdf = pd.merge(mps, myqlm, on=["PauliStrings"],how="outer", indicator=True)
pdf = pdf.dropna()
test = np.isclose(pdf["PauliCoefficients_x"], pdf["PauliCoefficients_y"]).all()

In [ ]:
myqlm

In [ ]:
for fail in to_test[1:]:
    mps_name = mps_folder + fail
    myqlm_name = myqlm_folder + fail
    mps, myqlm = compared_error(mps_name, myqlm_name, fail)
    pdf = pd.merge(
        mps, myqlm, on = ["PauliStrings"], how="outer", indicator=True
    )
    pdf = pdf[pdf["_merge"] != "both"]

In [ ]:
pdf

In [ ]:
to_test

In [ ]:
name = 'nqubits_30_depth_1_pauli.csv'

In [ ]:
mps_name = mps_folder + name
myqlm_name = myqlm_folder + name

In [ ]:
mps = pd.read_csv(mps_name, sep=";", index_col=0)
myqlm = pd.read_csv(myqlm_name, sep=";", index_col=0)  

In [ ]:
len(mps)

In [ ]:
len(myqlm)

In [ ]:
myqlm_name

In [ ]:
myqlm

In [ ]:
def compared_error(mps_name, myqlm_name, name):
    mps = pd.read_csv(mps_name, sep=";", index_col=0)
    myqlm = pd.read_csv(myqlm_name, sep=";", index_col=0)  
    return mps, myqlm

In [ ]:
pdf = pd.merge(
    mps, myqlm, on = ["PauliStrings"], how="outer", indicator=True
)

In [ ]:
pdf = pdf[pdf["_merge"] != "both"]

In [ ]:
left = pdf["PauliCoefficients_x"].dropna().abs().mean()
right = pdf["PauliCoefficients_y"].dropna().abs().mean()




if (left < 1.0e-15) and (right < 1.0e-15):
    print("Ol rait")
else:
    print("Uoh")

In [ ]:
right < 1.0e-15

In [ ]:
pd.concat([right, left], axis=0)

In [ ]:
left